In [1]:
from IPython.display import HTML
HTML(open('../style.css', 'r').read())

In [2]:
%load_ext nb_mypy

Version 1.0.6


In [3]:
Variable = str
Literal  = Variable | tuple[str, Variable]
Clause   = frozenset[Literal]

The <a href="https://en.wikipedia.org/wiki/Eight_queens_puzzle">eight queens puzzle</a> is the problem of placing eight chess queens on a chessboard so that no two queens can attack each other.  In <a href="https://en.wikipedia.org/wiki/Chess">chess</a> a queen can attack another piece if this piece is either
<ol>
    <li>in the same row,</li>
    <li>in the same column, or</li>
    <li>in the same diagonal.</li>
</ol>
The image below shows a queen in row 3, column 4.  All the locations where a piece can be captured by this queen are marked with an arrow.

<img src="queen-captures.png">

We will solve this puzzle by coding it as a formula of propositional logic.  This formula will be solvable iff the eight queens puzzle has a solution.  We will use the algorithm of *Davis and Putnam* to compute the solution of this formula.

In [4]:
def solve(Clauses: set[Clause]) -> set[Clause]: # forward declaration for type checker
    return None # type: ignore

In [5]:
%%capture
%run 07-Davis-Putnam-JW.ipynb

The function $\texttt{var}(r, c)$ takes a row $r$ and a column $c$ and returns the string $\texttt{'Q<}r\texttt{,}c\texttt{>'}$.  This string is interpreted as a propositional variable specifying that there is a queen in row $r$ and column $c$.  The image below shows how theses variables correspond to the positions on a chess board.

<img src="queens-vars.png">

The function `var(row, col)` takes two integers `row` and `col` as its argument and returns a string of the form `f'Q<{row},{col}>`.
This string is interpreted as a propositional variable.  This variable is `True` iff there is a queen in the given row and column on the board.

In [6]:
def var(row: int, col: int) -> Variable:
    return f'Q<{row},{col}>'

In [7]:
var(11,3)

'Q<11,3>'

Given a set of propositional variables $S$, the function $\texttt{atMostOne}(S)$ returns a set containing clauses that expresses the fact that **at most one** of the variables in $S$ is `True`.

In [8]:
def atMostOne(S: set[Variable]) -> set[Clause]: 
    return { frozenset({('¬', p), ('¬', q)}) for p in S
                                             for q in S 
                                             if  p < q 
           }

In [9]:
atMostOne({'a', 'b', 'c'})

{frozenset({('¬', 'b'), ('¬', 'c')}),
 frozenset({('¬', 'a'), ('¬', 'b')}),
 frozenset({('¬', 'a'), ('¬', 'c')})}

Given a `row` and the size of the board `n`, the procedure `atMostOneInRow(row, n)` computes a set of clauses that is `True` if and only there is at most one queen in the
given row.

In [10]:
def atMostOneInRow(row: int, n: int) -> set[Clause]:
    VarsInRow = { var(row, col) for col in range(1,n+1) }
    return atMostOne(VarsInRow)

In [11]:
atMostOneInRow(3, 4)

{frozenset({('¬', 'Q<3,1>'), ('¬', 'Q<3,2>')}),
 frozenset({('¬', 'Q<3,1>'), ('¬', 'Q<3,4>')}),
 frozenset({('¬', 'Q<3,1>'), ('¬', 'Q<3,3>')}),
 frozenset({('¬', 'Q<3,3>'), ('¬', 'Q<3,4>')}),
 frozenset({('¬', 'Q<3,2>'), ('¬', 'Q<3,4>')}),
 frozenset({('¬', 'Q<3,2>'), ('¬', 'Q<3,3>')})}

Given a column `col` and the size of the board `n`, the procedure `oneInColumn(col, n)` computes a set of clauses that is true if and only if there is at least one queen in the given column.

In [12]:
def oneInColumn(col: int, n: int) -> set[Clause]:
    VarsInColumn = { var(row, col) for row in range(1,n+1) }
    return { frozenset(VarsInColumn) }

In [13]:
oneInColumn(2, 4)

{frozenset({'Q<1,2>', 'Q<2,2>', 'Q<3,2>', 'Q<4,2>'})}

Given a number `k` and the size of the board `n`, the procedure `atMostOneInFallingDiagonal(k, n)` computes a set of clauses that is `True` if and only if there 
is at most one queen in the falling diagonal specified by the equation
```
     row - col = k.
```

In [14]:
def atMostOneInFallingDiagonal(k: int, n: int) -> set[Clause]:
    VarsInDiagonal = { var(row, col) for row in range(1, n+1)
                                     for col in range(1, n+1) 
                                     if  row - col == k 
                     }
    return atMostOne(VarsInDiagonal)

In [15]:
atMostOneInFallingDiagonal(0, 4)

{frozenset({('¬', 'Q<2,2>'), ('¬', 'Q<3,3>')}),
 frozenset({('¬', 'Q<2,2>'), ('¬', 'Q<4,4>')}),
 frozenset({('¬', 'Q<1,1>'), ('¬', 'Q<2,2>')}),
 frozenset({('¬', 'Q<1,1>'), ('¬', 'Q<3,3>')}),
 frozenset({('¬', 'Q<1,1>'), ('¬', 'Q<4,4>')}),
 frozenset({('¬', 'Q<3,3>'), ('¬', 'Q<4,4>')})}

Given a number `k` and the size of the board `n`, the procedure `atMostOneInRisingDiagonal(k, n)` computes a set of clauses that is `True` 
if and only if there is at most one queen in the rising diagonal specified by the equation
```
    row + col = k. 
```

In [16]:
def atMostOneInRisingDiagonal(k: int, n: int) -> set[Clause]:
    VarsInDiagonal = { var(row, col) for row in range(1, n+1)
                                     for col in range(1, n+1) 
                                     if  row + col == k 
                     }
    return atMostOne(VarsInDiagonal)

In [17]:
atMostOneInRisingDiagonal(5, 4)

{frozenset({('¬', 'Q<1,4>'), ('¬', 'Q<2,3>')}),
 frozenset({('¬', 'Q<3,2>'), ('¬', 'Q<4,1>')}),
 frozenset({('¬', 'Q<2,3>'), ('¬', 'Q<4,1>')}),
 frozenset({('¬', 'Q<1,4>'), ('¬', 'Q<3,2>')}),
 frozenset({('¬', 'Q<2,3>'), ('¬', 'Q<3,2>')}),
 frozenset({('¬', 'Q<1,4>'), ('¬', 'Q<4,1>')})}

The function `allClauses(n)` takes the size of the board $n$ and computes a set of clauses that specify that
* there is at most one queen in every row,
* there is at most one queen in every rising diagonal,
* there is at most one queen in every falling diagonal, and
* there is at least one queen in every column.

In [18]:
def allClauses(n: int) -> set[Clause]:
    All = [ atMostOneInRow(row, n)           for row in range(1, n+1)        ] \
        + [ atMostOneInRisingDiagonal(k, n)  for k in range(3, (2*n-1)+1)    ] \
        + [ atMostOneInFallingDiagonal(k, n) for k in range(-(n-2), (n-2)+1) ] \
        + [ oneInColumn(col, n)              for col in range(1, n+1)        ]
    return { clause for S in All for clause in S }

In [19]:
for C in allClauses(16):
    print(set(C))

{('¬', 'Q<2,2>'), ('¬', 'Q<9,9>')}
{('¬', 'Q<9,9>'), ('¬', 'Q<9,16>')}
{('¬', 'Q<16,13>'), ('¬', 'Q<14,11>')}
{('¬', 'Q<2,11>'), ('¬', 'Q<8,5>')}
{('¬', 'Q<8,5>'), ('¬', 'Q<8,2>')}
{('¬', 'Q<14,2>'), ('¬', 'Q<9,7>')}
{('¬', 'Q<5,7>'), ('¬', 'Q<6,6>')}
{('¬', 'Q<2,11>'), ('¬', 'Q<9,4>')}
{('¬', 'Q<15,15>'), ('¬', 'Q<11,11>')}
{('¬', 'Q<4,12>'), ('¬', 'Q<4,13>')}
{('¬', 'Q<7,2>'), ('¬', 'Q<7,14>')}
{('¬', 'Q<8,15>'), ('¬', 'Q<8,13>')}
{('¬', 'Q<10,2>'), ('¬', 'Q<3,9>')}
{('¬', 'Q<7,3>'), ('¬', 'Q<7,5>')}
{('¬', 'Q<7,6>'), ('¬', 'Q<5,8>')}
{('¬', 'Q<9,4>'), ('¬', 'Q<4,9>')}
{('¬', 'Q<14,12>'), ('¬', 'Q<14,13>')}
{('¬', 'Q<2,11>'), ('¬', 'Q<10,3>')}
{('¬', 'Q<4,2>'), ('¬', 'Q<4,12>')}
{('¬', 'Q<14,2>'), ('¬', 'Q<14,4>')}
{('¬', 'Q<8,12>'), ('¬', 'Q<10,10>')}
{('¬', 'Q<13,3>'), ('¬', 'Q<14,4>')}
{('¬', 'Q<12,7>'), ('¬', 'Q<12,12>')}
{('¬', 'Q<11,9>'), ('¬', 'Q<11,12>')}
{('¬', 'Q<14,14>'), ('¬', 'Q<14,13>')}
{('¬', 'Q<14,14>'), ('¬', 'Q<14,1>')}
{('¬', 'Q<4,5>'), ('¬', 'Q<4,11>')}
{('¬', 'Q

The set of all clauses contains 512 clauses.  There are 64 variables.  

In [20]:
len(allClauses(16))

4416

The function $\texttt{queens}(n)$ solves the *n queens problem*.

In [21]:
def queens(n: int) -> set[Clause] | None:
    "Solve the n queens problem."
    Clauses  = allClauses(n)
    Solution = solve(Clauses)
    if Solution != { frozenset() }:
        return Solution
    else:
        print(f'The problem is not solvable for {n} queens!')
        return None

In [22]:
Solution: set[Clause]

The *8 queens problem* can be solved in less than a tenth of a second using the 
pure *Davis Putnam algorithm* that does not use the Jereslow-Wang heuristic. If we want to solve the *16 queens problem*, we need to use the Jereslow-Wang heuristic.  With this heuristic, the *16 queens problem* is then solved in roughly 4 seconds on my Mac Studio from 2023.

In [24]:
%%time
Solution = queens(16)
Solution

CPU times: user 4.88 s, sys: 30.7 ms, total: 4.91 s
Wall time: 4.91 s


{frozenset({('¬', 'Q<11,7>')}),
 frozenset({('¬', 'Q<5,3>')}),
 frozenset({('¬', 'Q<14,8>')}),
 frozenset({('¬', 'Q<8,12>')}),
 frozenset({('¬', 'Q<11,1>')}),
 frozenset({('¬', 'Q<12,1>')}),
 frozenset({'Q<14,4>'}),
 frozenset({('¬', 'Q<10,3>')}),
 frozenset({'Q<7,2>'}),
 frozenset({('¬', 'Q<10,2>')}),
 frozenset({('¬', 'Q<14,10>')}),
 frozenset({('¬', 'Q<9,10>')}),
 frozenset({('¬', 'Q<9,4>')}),
 frozenset({('¬', 'Q<16,1>')}),
 frozenset({('¬', 'Q<11,9>')}),
 frozenset({('¬', 'Q<16,10>')}),
 frozenset({('¬', 'Q<7,3>')}),
 frozenset({('¬', 'Q<7,11>')}),
 frozenset({('¬', 'Q<16,13>')}),
 frozenset({('¬', 'Q<1,15>')}),
 frozenset({('¬', 'Q<13,3>')}),
 frozenset({('¬', 'Q<10,15>')}),
 frozenset({('¬', 'Q<2,11>')}),
 frozenset({('¬', 'Q<11,13>')}),
 frozenset({('¬', 'Q<2,16>')}),
 frozenset({('¬', 'Q<12,16>')}),
 frozenset({('¬', 'Q<6,13>')}),
 frozenset({('¬', 'Q<14,2>')}),
 frozenset({('¬', 'Q<2,1>')}),
 frozenset({('¬', 'Q<3,16>')}),
 frozenset({('¬', 'Q<9,1>')}),
 frozenset({('¬', 'Q<1

# Visualization

The following code requires that you install `chess-problem-visuals`, which is a package provided by *Philipp Polland*.  This package can be installed into the environment  with the following command:
```
pip install git+https://github.com/reclinarka/chess-problem-visuals
```
The following line performs this installation inside the jupyter notebook.
Once you have executed this line once, you can comment it.

In [ ]:
!pip install git+https://github.com/reclinarka/chess-problem-visuals

Next, we have to import the package `chess_problem_visuals` that was installed in the previous cell.

In [ ]:
import chess_problem_visuals as cpv

The function `remove_negative_literals` takes one input:
* `Solution` is a set of unit clauses.

The function returns the set of all those unit clauses in `Solution` that do not contain negative literals.

In [ ]:
def remove_negative_literals(Solution):
    Result = set()
    for Clause in Solution:
        for literal in Clause:
            if isinstance(literal, str):
                Result.add(literal)
    return Result

The function `extract_row_col` takes one argument:
* `var_name` is a string of the form `f'Q<{row},{col}>'`.

It returns the pair `row, col`.

In [ ]:
remove_negative_literals({ frozenset({'p'}), frozenset({('¬', 'q')}), frozenset({('¬', 'r')}), frozenset({'s'}) })

In [ ]:
def extract_row_col(var_name):
    left  = var_name.find('<')
    comma = var_name.find(',')
    right = var_name.find('>')
    row   = var_name[left+ 1:comma]
    col   = var_name[comma+1:right]
    return row, col

In [ ]:
extract_row_col('Q<13,9>')

In [ ]:
def transform(Solution):
    Solution = remove_negative_literals(Solution)
    Result   = {}
    for name in Solution:
        row, col = extract_row_col(name)
        Result[int(row)] = int(col)
    return Result

The function `show_solution(Solution, width)` takes a dictionary that contains a variable assignment that represents a solution to the n queens puzzle.  It displays this Solution on a chess board.
* `Solution` is a dictionary mapping the variables $\texttt{V}_i$ to numbers.
  If $\texttt{Solution[V}_i\texttt{]} = k$, then the queen in row $i$ is placed in column $k$.
* `width` specifies the size of the board as a percentage of the width of notebook.  

In [ ]:
def show_solution(Solution, width="50%"):
    Solution = transform(Solution)
    n = len(Solution)
    Queens = [None for col in range(n)]
    board  = cpv.Board(n, html_width=width)
    for row in range(n):
        col = Solution.get(row+1, None)
        if col != None:
            board.add_piece((col-1, row), 'Q')
    return board

In [ ]:
Solution 

In [ ]:
show_solution(Solution, "50%")